In [11]:
from readlif.reader import LifFile
import numpy as np
from tifffile import imwrite
import os
import pandas as pd
import numpy as np
import tifffile
import re

# type lif path, will display all stack names
# type desired stack name, if correct will display metadata
lif_path = r"D:\20240526_802confocal_penetration 1\20240526 penetration.lif"
stack_name = 'sample 4_Merging_001'
safe_name = re.sub(r'[<>:"/\\|?*]', '', stack_name)
out_folder = r"C:\Users\bear\Desktop\figure 1"
lif = LifFile(lif_path)

match = 0
for idx, img in enumerate(lif.image_list):
    print(img['name'])
    if img['name'] == stack_name:
        stack = img
        stack_idx = idx
        match = 1
print()

if match == 1:
    for key, value in stack.items():
        if key != 'settings':
            print(f"{key}: {value}")

sample 1
sample 1_Merging_001
sample 2
sample 2_Merging_001
sample 3
sample 3_Merging_001
sample 4
sample 4_Merging_001
sample 5
sample 5_Merging_001
sample 5 bottom
sample 5 bottom_Merging_001
sample 6
sample 6_Merging_001
sample 6 again
sample 6 again_Merging_001
sample 14
sample 14_Merging_001
sample 15
sample 15_Merging_001
sample 16
sample 16_Merging_001
sample 17
sample 17_Merging_001
sample 10
sample 10_Merging_001
sample 11
sample 11_Merging_001
sample 12
sample 12_Merging_001
sample 13
sample 13_Merging_001

dims: Dims(x=7482, y=7458, z=1, t=1, m=1)
display_dims: (1, 2)
dims_n: {1: 7482, 2: 7458, 3: 1}
scale_n: {1: 1.761249773987464, 2: 1.7612490720789469, 3: None}
path: 20240526 penetration.lif/
name: sample 4_Merging_001
channels: 1
scale: (1.761249773987464, 1.7612490720789469, None, None)
bit_depth: (16,)
mosaic_position: []
channel_as_second_dim: False


In [12]:
# store lif file name, stack name, xy pixel size and z pixel size in csv
# not automatic to loop over all files, instead use it when needed
metadata = {
    'folder': os.path.basename(os.path.dirname(lif_path)),
    'lif_name': os.path.basename(lif_path),
    'stack_name': stack_name,
    'num_channels': stack['channels'],
    'pixel_size_xy': 1/stack['scale'][0],  # microns/pixel
    'pixel_size_z': abs(1/stack['scale'][2]) if stack['scale'][2] is not None else None,   # microns/pixel
    'bit_depth': stack['bit_depth'][0]
}

csv_path = 'metadata.csv'
if os.path.exists(csv_path):
    df_existing = pd.read_csv(csv_path)
    df_new = pd.concat([df_existing, pd.DataFrame([metadata])], ignore_index=True)
    df_new = df_new.drop_duplicates(subset=['folder', 'lif_name', 'stack_name'], keep='last')
    df_new.to_csv(csv_path, index=False)
else:
    pd.DataFrame([metadata]).to_csv(csv_path, index=False)

In [13]:
# output tif (multichannel)
safe_name = re.sub(r'[<>:"/\\|?*]', '', stack_name)
tif_name = f"{safe_name}.tif"
tif_path = os.path.join(out_folder, tif_name)
stack_img = lif.get_image(stack_idx)
num_z, num_c, num_y, num_x, bit_depth = stack_img.dims.z, stack_img.channels, stack_img.dims.y, stack_img.dims.x, stack_img.bit_depth[0]
if bit_depth == 8:
    dtype = np.uint8
elif bit_depth == 16:
    dtype = np.uint16
    
tif_stack = np.zeros((num_z, num_c, num_y, num_x), dtype=dtype)
for z in range(num_z):
    for c in range(num_c):
        tif_stack[z, c] = stack_img.get_frame(z=z, t=0, c=c)
    if (z + 1) % 50 == 0:
        print(f'plane {z + 1} finished')

imwrite(tif_path, tif_stack, imagej = True, metadata = {'axes': 'ZCYX'})

In [6]:
# reuse python matlab to convert to single channel
import os
import tifffile

multichannel_path = os.path.join(out_folder, f"{safe_name}.tif")

folder, filename = os.path.split(multichannel_path)
base, ext = os.path.splitext(filename)
multichannel = tifffile.imread(multichannel_path)
z, c, y, x = multichannel.shape
assert c == min(z, c, y, x)

for i in range(c):
    channel = multichannel[:, i, :, :]
    dst = os.path.join(folder, f"{base}_c{i}{ext}")
    tifffile.imwrite(dst, channel, imagej = True, metadata = {'axes': 'ZYX'})

ValueError: not enough values to unpack (expected 4, got 2)